In [38]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from pathlib import Path
from math import acos, degrees

In [39]:
def get_angle(dot1, dot2, dot3):
    if not (dot1 and dot2 and dot3):
        return 0
    a = (dot2.x - dot1.x, dot2.y - dot1.y, dot2.z - dot1.z)
    b = (dot2.x - dot3.x, dot2.y - dot3.y, dot2.z - dot3.z)
    skalar_mul = a[0] * b[0] + a[1] * b[1]
    norm_mul = ((a[0] ** 2 + a[1] ** 2) * (b[0] ** 2 + b[1] ** 2)) ** 0.5
    return degrees(acos(skalar_mul / norm_mul))

In [40]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def get_data_from_file(path, value, df):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)
    if results.pose_landmarks:
        landmarks = {'image_path': str(file_path)}
        
        for i, landmark in enumerate(results.pose_landmarks.landmark):
            landmarks[f'x_{i}'] = landmark.x
            landmarks[f'y_{i}'] = landmark.y
            landmarks[f'z_{i}'] = landmark.z
            landmarks[f'visibility_{i}'] = landmark.visibility
        
        landmarks['left_knee_angle'] = get_angle(results.pose_landmarks.landmark[23], 
                                                 results.pose_landmarks.landmark[25], 
                                                 results.pose_landmarks.landmark[27])
                    
        landmarks['right_knee_angle'] = get_angle(results.pose_landmarks.landmark[24], 
                                                  results.pose_landmarks.landmark[26], 
                                                  results.pose_landmarks.landmark[28])
        
        landmarks['label'] = value
        
        df = df._append(landmarks, ignore_index=True)
    return df

In [41]:
classes = {
    'bent': 0,
    'straight': 1
}

root = 'legs_pics/'

df = pd.DataFrame()
for key, value in classes.items():
    path = Path(root + key)
    for file_path in path.glob('*'):
        df = get_data_from_file(str(file_path), value, df)

In [42]:
X = np.array(df.iloc[:, 1:-1])
y = np.array(df.iloc[:, -1])

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(random_state=42, max_iter=500).fit(X_train, y_train)

y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

print(f'test accuracy: {accuracy_score(y_test, y_test_pred)}\ntrain accuracy: {accuracy_score(y_train, y_train_pred)}')

test accuracy: 1.0
train accuracy: 1.0
